In [2]:
import pandas as pd
import psycopg2

# Load your cleaned CSV
df = pd.read_csv(r"C:\Users\bezis\Downloads\GooglePlayScraper\google-play-app-reviews-analysis\notebooks\task2_sentiment_themes.csv")

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",       # your username
    password="beza091163",  # your password
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Insert unique banks
for bank_name, source in df[['bank', 'source']].drop_duplicates().values:
    cur.execute(
        "INSERT INTO banks (bank_name, app_name) VALUES (%s, %s) ON CONFLICT DO NOTHING;",
        (bank_name, source)
    )

# Insert reviews
for row in df.itertuples():
    cur.execute(
        """INSERT INTO reviews
           (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, source, nouns, identified_theme, noun_text)
           VALUES (
               (SELECT bank_id FROM banks WHERE bank_name=%s),
               %s, %s, %s, %s, %s, %s, %s, %s, %s
           );""",
        (row.bank, row.review, row.rating, row.date, row.sentiment_label,
         row.sentiment_score, row.source, row.nouns, row.identified_theme, row.noun_text)
    )

# Commit and close connection
conn.commit()
cur.close()
conn.close()

print("CSV data inserted successfully!")


CSV data inserted successfully!
